In [22]:
import requests
import json
import pandas as pd
from dateutil import parser
import os
from dotenv import load_dotenv
load_dotenv()


True

In [23]:
OANDA_API_KEY = os.environ["OANDA_API_KEY"]
OANDA_URL = os.environ.get(
    "OANDA_URL",
    "https://api-fxpractice.oanda.com/v3/instruments/{}/candles"
)
OANDA_ACCOUNT_ID = os.environ["OANDA_ACCOUNT_ID"]

In [24]:
session = requests.Session()

In [25]:
session.headers.update({
    "Authorization": "Bearer " + OANDA_API_KEY,
    "Content-Type": "application/json"
})

In [26]:
params = {
     
    "count": 10,
    "granularity": "H1",
    "price": "MBA"
    
   
}

In [27]:
url = f"{OANDA_URL}/accounts/{OANDA_ACCOUNT_ID}/instruments"

In [28]:
response = session.get(url, params=params, data=None, headers=None)

In [29]:
response.status_code

200

In [30]:
data = response.json()


In [31]:
instruments_list = data['instruments']

In [32]:
len(instruments_list)

127

In [33]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [34]:
key_interest = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [35]:
instruments_dict = {}
for instrument in instruments_list:
    key = instrument['name']
    instruments_dict[key] = {k: instrument[k] for k in key_interest}

In [36]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [37]:
with open('instruments.json', 'w') as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [38]:
def fetch_candles(pair_name, granularity="M5", count=5000):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = {
        "granularity": granularity,
        "count": count,
        "price": "MBA"
    }
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    
   

In [39]:
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for price in prices:
            for field in ohlc:
                key = f"{price}_{field}"
                new_dict[key] = float(candle[price][field])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, granularity="H1", count=10):
    status_code, data = fetch_candles(pair_name, granularity, count)
    if status_code != 200:
        print(f"Error fetching data for {pair_name}: {status_code}")
        return
    if len(data) == 0:
        print(f"No data returned for {pair_name}")
        return
    candles_df = get_candles_df(data)
    file_name = f"data/{pair_name}_{granularity}.pkl"
    return candles_df.to_pickle(file_name)


In [40]:
create_data_file("EUR_USD", "H1", 100)
code, data = fetch_candles("EUR_USD", granularity="H1", count=20)
candles_df = get_candles_df(data)   
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2025-10-22 04:00:00+00:00,2456,1.16072,1.16096,1.16050,1.16092,1.16063,1.16089,1.16041,1.16084,1.16080,1.16104,1.16058,1.16101
1,2025-10-22 05:00:00+00:00,2967,1.16092,1.16158,1.16090,1.16136,1.16083,1.16151,1.16082,1.16129,1.16101,1.16166,1.16099,1.16144
2,2025-10-22 06:00:00+00:00,10440,1.16136,1.16150,1.16042,1.16096,1.16128,1.16142,1.16034,1.16088,1.16145,1.16158,1.16049,1.16103
3,2025-10-22 07:00:00+00:00,9234,1.16096,1.16129,1.15996,1.16008,1.16088,1.16121,1.15987,1.16000,1.16104,1.16137,1.16004,1.16017
4,2025-10-22 08:00:00+00:00,10715,1.16007,1.16008,1.15898,1.15976,1.15998,1.15999,1.15889,1.15968,1.16016,1.16017,1.15906,1.15985
5,2025-10-22 09:00:00+00:00,6000,1.15976,1.16021,1.15924,1.15946,1.15967,1.16013,1.15915,1.15938,1.15985,1.16030,1.15933,1.15954
6,2025-10-22 10:00:00+00:00,5391,1.15946,1.15970,1.15828,1.15858,1.15937,1.15961,1.15820,1.15850,1.15954,1.15979,1.15837,1.15867
7,2025-10-22 11:00:00+00:00,7801,1.15858,1.15892,1.15824,1.15825,1.15850,1.15884,1.15816,1.15817,1.15866,1.15901,1.15832,1.15833
8,2025-10-22 12:00:00+00:00,10064,1.15824,1.15917,1.15770,1.15830,1.15815,1.15910,1.15762,1.15823,1.15832,1.15924,1.15778,1.15838
9,2025-10-22 13:00:00+00:00,11165,1.15831,1.15977,1.15820,1.15950,1.15823,1.15969,1.15812,1.15942,1.15839,1.15985,1.15828,1.15957


In [41]:
our_currencies = ['EUR', 'USD', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'NZD']
instruments_dict.keys()


dict_keys(['TRY_JPY', 'AUD_JPY', 'USB02Y_USD', 'XAU_USD', 'USD_CNH', 'NZD_JPY', 'EUR_GBP', 'XAG_GBP', 'XAU_NZD', 'CHF_HKD', 'CN50_USD', 'USD_CZK', 'US30_USD', 'NZD_HKD', 'JP225Y_JPY', 'EUR_NOK', 'USD_CAD', 'EUR_AUD', 'EUR_SGD', 'USD_HKD', 'CH20_CHF', 'CAD_HKD', 'XAG_CHF', 'USD_CHF', 'XAG_HKD', 'AUD_HKD', 'ESPIX_EUR', 'NZD_CHF', 'AUD_CHF', 'GBP_CHF', 'USD_THB', 'XAU_JPY', 'XAU_HKD', 'EUR_HKD', 'CHF_JPY', 'GBP_HKD', 'EUR_NZD', 'XAG_AUD', 'WTICO_USD', 'XAG_NZD', 'AUD_SGD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'BTC_USD', 'SGD_JPY', 'GBP_ZAR', 'XAG_JPY', 'ETH_USD', 'ZAR_JPY', 'USD_SEK', 'GBP_SGD', 'XAU_SGD', 'CAD_CHF', 'US2000_USD', 'SPX500_USD', 'AUD_NZD', 'USB30Y_USD', 'BCO_USD', 'HKD_JPY', 'XAU_XAG', 'HK33_HKD', 'USB10Y_USD', 'EU50_EUR', 'USD_NOK', 'XAU_GBP', 'GBP_AUD', 'USD_PLN', 'XCU_USD', 'EUR_ZAR', 'NZD_USD', 'LTC_USD', 'XAG_EUR', 'DE30_EUR', 'USD_ZAR', 'CAD_JPY', 'CAD_SGD', 'USD_HUF', 'EUR_CAD', 'NATGAS_USD', 'FR40_EUR', 'DE10YB_EUR', 'CHF_ZAR', 'USD_DKK', 'NAS100_USD', 'XAU_CAD', 'XPD_

In [42]:
for pair_1 in our_currencies:
    for pair_2 in our_currencies:
        if pair_1 == pair_2:
            continue
        pair_name = f"{pair_1}_{pair_2}"
        if pair_name in instruments_dict:
            for g in ["H1", "H4"]:
                print(f"Creating data file for {pair_name}_{g}")
                create_data_file(pair_name, granularity=g, count=4001)

Creating data file for EUR_USD_H1
Creating data file for EUR_USD_H4
Creating data file for EUR_GBP_H1
Creating data file for EUR_GBP_H4
Creating data file for EUR_JPY_H1
Creating data file for EUR_JPY_H4
Creating data file for EUR_AUD_H1
Creating data file for EUR_AUD_H4
Creating data file for EUR_CAD_H1
Creating data file for EUR_CAD_H4
Creating data file for EUR_CHF_H1
Creating data file for EUR_CHF_H4
Creating data file for EUR_NZD_H1
Creating data file for EUR_NZD_H4
Creating data file for USD_JPY_H1
Creating data file for USD_JPY_H4
Creating data file for USD_CAD_H1
Creating data file for USD_CAD_H4
Creating data file for USD_CHF_H1
Creating data file for USD_CHF_H4
Creating data file for GBP_USD_H1
Creating data file for GBP_USD_H4
Creating data file for GBP_JPY_H1
Creating data file for GBP_JPY_H4
Creating data file for GBP_AUD_H1
Creating data file for GBP_AUD_H4
Creating data file for GBP_CAD_H1
Creating data file for GBP_CAD_H4
Creating data file for GBP_CHF_H1
Creating data 